In [1]:
! pip install trec-car-tools transformers
import torch
import nltk
from nltk.tokenize import sent_tokenize
from torch.nn import functional, CosineEmbeddingLoss, BCEWithLogitsLoss, CosineSimilarity, BCELoss
from torch.utils.data.dataset import Dataset
from trec_car import read_data
from cbor import *
from transformers import BertModel, BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments
import random
model = BertModel.from_pretrained('bert-base-uncased')
nspmodel = BertForNextSentencePrediction.from_pretrained('bert-base-uncased') #output_hidden_states=True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
file = 'train.pages.cbor'
nltk.download('punkt')

     |████████████████████████████████| 2.3MB 7.4MB/s 
     |████████████████████████████████| 3.3MB 47.4MB/s 
     |████████████████████████████████| 901kB 43.5MB/s 
  Created wheel for cbor: filename=cbor-1.0.0-cp37-cp37m-linux_x86_64.whl size=51258 sha256=8d215e2d0354e939bc418ea3f43fc68d28930549ac3a32646a29321eedfcb3b6
  Stored in directory: /root/.cache/pip/wheels/40/5c/a5/e6d629446a6a687ba328c55f1589234c29b99ef35b1a65dbaa
Successfully built cbor


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerN


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def generate_wrong_next(corpus):
  # helper function
  # given an array of paragraphs, generate wrong next paras
  # cant do completely random as there is a random chance to pick next para using shuffle()
  # which i want to avoid
  wrong_next = []
  second_sentences = corpus[1:len(corpus)]
  for i in range(len(second_sentences)):
    temp = second_sentences.copy()
    temp.pop(i)
    next = random.choice(temp)
    wrong_next.append(next)
  return wrong_next

In [ ]:
# small test for data set generation

labels = []
first_input_ids = torch.empty(0)
second_input_ids = torch.empty(0)
pages = read_data.iter_pages(open(file, 'rb'))
count = 0
pagecount = 0
for page in pages:
    pagecount += 1
    corpus = page.get_text().split('\n')
    count += len(corpus)

    first_paragraphs = corpus[0:len(corpus) - 1]
    second_paragraphs = corpus[1:len(corpus)]
    wrong_next_paragraphs = generate_wrong_next(corpus)

    
    first_sentences = []
    second_sentences = []
    wrong_next_sentences = []



    for i in range(len(first_paragraphs)):
      last_para = sent_tokenize(first_paragraphs[i])
      first_sentences.append(last_para[len(last_para) - 1]) if len(last_para) > 0 else None

      first_para = sent_tokenize(second_paragraphs[i])
      second_sentences.append(first_para[0]) if len(first_para) > 0 else None

      wrong_para = sent_tokenize(wrong_next_paragraphs[i])
      wrong_next_sentences.append(wrong_para[0]) if len(wrong_para) > 0 else None

    print(first_sentences[0])

    first_encoding = tokenizer(first_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
    second_encoding = tokenizer.encode(second_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
    wrong_encoding = tokenizer.encode(wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
    
    print(first_encoding)
    break

    p_labels = [1.] * first_encoding['input_ids'].size()
    n_labels = [-1.] * first_encoding['input_ids'].size()
    labels += p_labels + n_labels

    # good = torch.tensor(first_encoding, second_encoding)
    # bad = torch.tensor(first_encoding, wrong_encoding)

    p_encodings = torch.cat((first_encoding['input_ids'], first_encoding['input_ids']), 0)
    n_encodings = torch.cat((second_encoding['input_ids'], wrong_encoding['input_ids']), 0)


    # encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
    first_input_ids = torch.cat((p_encodings, first_input_ids), 0)
    second_input_ids = torch.cat((n_encodings, second_input_ids), 0)

"""
print(first_input_ids.size())
print(second_input_ids.size())
print(len(labels))
print(count) # number of pages
print(pagecount)
print(len(wrong_next_sentences))
print(len(first_sentences))
"""


testmodel = BertModel.from_pretrained('bert-base-uncased')
outputs = testmodel(first_encoding['input_ids'])
print(outputs['last_hidden_state'][:, 0, :])

In [3]:
class BertParaDataSet(Dataset):
  """
  Generates a dataset of pairs of paragraphs.
  [query, true_next_paragraph, wrong_next_para]
  
  """
  def __init__(self, file):

    self.file = file
    self.labels = []
    self.first_input_ids = torch.empty(0)
    self.second_input_ids = torch.empty(0)


    pages = read_data.iter_pages(open(file, 'rb'))
    for page in pages:
      corpus = page.get_text().split('\n')

      first_paragraphs = corpus[0:len(corpus) - 1]
      second_paragraphs = corpus[1:len(corpus)]
      wrong_next_paragraphs = generate_wrong_next(corpus)
        

      first_encoding = tokenizer(first_paragraphs, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
      second_encoding = tokenizer(second_paragraphs, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
      wrong_encoding = tokenizer(wrong_next_paragraphs, return_tensors='pt', padding='max_length', max_length=512, truncation=True)

      p_labels = [1.] * first_encoding['input_ids'].size()[0]
      n_labels = [-1.] * first_encoding['input_ids'].size()[0]
      self.labels += p_labels + n_labels

      
      p_encodings = torch.cat((first_encoding['input_ids'], first_encoding['input_ids']), 0)
      n_encodings = torch.cat((second_encoding['input_ids'], wrong_encoding['input_ids']), 0)

      self.first_input_ids = torch.cat((p_encodings, self.first_input_ids), 0)
      self.second_input_ids = torch.cat((n_encodings, self.second_input_ids), 0)

    
  def __len__(self):
    return self.first_input_ids.size()[0]

  def __getitem__(self, index):
    # label as 1 because we want the the cosine to be close to 1 for these following sentences
    sample = {'first_input_ids': self.first_input_ids[index].type(torch.LongTensor), 'second_input_ids':  self.second_input_ids[index].type(torch.LongTensor), 'label': self.labels[index]}
    return sample


In [ ]:
class TwoSentenceDataSet(Dataset):
  """
  Generates a dataset of pairs of paragraph pairs using 
  the last sentence of the query, and the first sentence
   of the following paragraph.
  [last sentence of query paragraph, first sentence of following paragraph]

  """
  def __init__(self, file):
    self.file = file
    self.labels = []
    self.first_input_ids = torch.empty(0)
    self.second_input_ids = torch.empty(0)

    pages = read_data.iter_pages(open(file, 'rb'))
    for page in pages:
      corpus = page.get_text().split('\n')

      first_paragraphs = corpus[0:len(corpus) - 1]
      second_paragraphs = corpus[1:len(corpus)]
      wrong_next_paragraphs = generate_wrong_next(corpus)

      first_sentences = []
      second_sentences = []
      wrong_next_sentences = []

      for i in range(len(first_paragraphs)):
        last_para = sent_tokenize(first_paragraphs[i])
        first_sentences.append(last_para[len(last_para) - 1]) if len(last_para) > 0 else None

        first_para = sent_tokenize(second_paragraphs[i])
        second_sentences.append(first_para[0]) if len(first_para) > 0 else None

        wrong_para = sent_tokenize(wrong_next_paragraphs[i])
        wrong_next_sentences.append(wrong_para[0]) if len(wrong_para) > 0 else None
        

      first_encoding = tokenizer.encode(first_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      second_encoding = tokenizer.encode(second_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      wrong_encoding = tokenizer.encode(wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

      p_labels = [1.] * first_encoding['input_ids'].size()[0]
      n_labels = [-1.] * first_encoding['input_ids'].size()[0]
      self.labels += p_labels + n_labels

      # good = torch.tensor(first_encoding, second_encoding)
      # bad = torch.tensor(first_encoding, wrong_encoding)

      p_encodings = torch.cat((first_encoding['input_ids'], first_encoding['input_ids']), 0)
      n_encodings = torch.cat((second_encoding['input_ids'], wrong_encoding['input_ids']), 0)


      # encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
      self.first_input_ids = torch.cat((p_encodings, self.first_input_ids), 0)
      self.second_input_ids = torch.cat((n_encodings, self.second_input_ids), 0)

    
  def __len__(self):
    return self.first_input_ids.size()[0]

  def __getitem__(self, index):
    # label as 1 because we want the the cosine to be close to 1 for these following sentences
    sample = {'first_input_ids': self.first_input_ids[index].type(torch.LongTensor), 'second_input_ids':  self.second_input_ids[index].type(torch.LongTensor), 'label': self.labels[index]}
    return sample


In [ ]:
class BertFNSDataSet(Dataset):
  """
  BertForNextSentence
  Generates a dataset of pairs of paragraphs.
  [paragraph, following paragraph]

  """
  def __init__(self, file):
    self.file = file
    self.labels = []
    self.input_ids = torch.empty(0)

    pages = read_data.iter_pages(open(file, 'rb'))
    for page in pages:
      corpus = page.get_text().split('\n')

      first_sentences = corpus[0:len(corpus) - 1]
      second_sentences = corpus[1:len(corpus)]
      wrong_next_sentences = generate_wrong_next(corpus)

      p_encoding = tokenizer(first_sentences, second_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      p_labels = [1] * p_encoding['input_ids'].size()[0]
      n_encoding = tokenizer(first_sentences, wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      n_labels = [0] * n_encoding['input_ids'].size()[0]
      self.labels += p_labels + n_labels
      encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
      self.input_ids = torch.cat((encoding_cat, self.input_ids), 0)

    
  def __len__(self):
    return self.input_ids.size()[0]

  def __getitem__(self, index):
    sample = {'input_ids': self.input_ids[index].type(torch.LongTensor), 'label': self.labels[index]}
    return sample


In [ ]:
class CosineSimTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    query_text = inputs['first_input_ids']
    second_text = inputs['second_input_ids']

    # taking the mean of the last layer
    first_logits = model(query_text)['last_hidden_state'][:, 0, :]
    second_logits = model(second_text)['last_hidden_state'][:, 0, :]
    # outputs = torch.cat((first_logits, second_logits), 0)
    outputs = {
        "first_logits": first_logits,
        "second_logits": second_logits
    }
    cos = CosineSimilarity()
    c = cos(first_logits, second_logits)

    lf = BCEWithLogitsLoss()
    loss = lf(c, labels)
    
    # sim = cos(first_logits, second_logits)
    return (loss, outputs) if return_outputs else loss



In [5]:
class CosineEmbTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    query_text = inputs['first_input_ids']
    second_text = inputs['second_input_ids']

    # logits of the CLS token for each sentence/para
    first_logits = model(query_text)['last_hidden_state'][:, 0, :]
    second_logits = model(second_text)['last_hidden_state'][:, 0, :]

    # outputs = torch.cat((first_logits, second_logits), 0)
    outputs = {
        "first_logits": first_logits,
        "second_logits": second_logits
    }
    cos = CosineEmbeddingLoss()
    loss = cos(first_logits, second_logits, labels)
    
    # sim = cos(first_logits, second_logits)
    return (loss, outputs) if return_outputs else loss

In [7]:
ds = BertParaDataSet(file)
ds_len = ds.__len__()
print(ds_len)
# train_len = int(round(0.8 * ds_len))
# eval_len = int(round(0.2 * ds_len))
# train_dataset, eval_dataset = torch.utils.data.random_split(ds, [train_len, eval_len])
# print(train_len)

11858


In [9]:
training_args = TrainingArguments(
    output_dir='./drive/MyDrive/thesis_models/bert/batch4-emblength512-CLSonly',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    save_strategy='epoch'
)



trainer = CosineEmbTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset = ds,
)


In [ ]:
# model.from_pretrained('./drive/MyDrive/thesis_models/bert/cosine_emb_bY1_batch16/checkpoint-593')

In [1]:
trainer.train()

NameError: ignored

In [ ]:
 trainer.evaluate()
# model.save_pretrained('./drive/MyDrive/thesis_models/bert')
# nspmodel.save_pretrained('./drive/MyDrive/thesis_models/bertfnsp')